In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data import load_data

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from tensorflow.keras import utils
from tensorflow.keras import losses
from tensorflow.keras import backend as K

In [42]:
import importlib

In [37]:
import sys
sys.path.append("../../library")

In [45]:
import utils

In [29]:
class MTLDNN:
    def __init__(self, ni, params):
        self.model = MTLDNN.build_model(ni, params)
        
    def set_seed(s):
        # 0. clear backend session
        K.clear_session()
        
        # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
        import os
        os.environ['PYTHONHASHSEED']=str(s)

        # 2. Set `python` built-in pseudo-random generator at a fixed value
        import random
        random.seed(s)

        # 3. Set `numpy` pseudo-random generator at a fixed value
        import numpy as np
        np.random.seed(s)

        # 4. Set the `tensorflow` pseudo-random generator at a fixed value
        import tensorflow as tf
        tf.random.set_seed(s)
        # for later versions: 
        # tf.compat.v1.set_random_seed(seed_value)

    def build_model(ni, params):
        act = params.get('act')
        n, n1, n2, n3 = params.get('arq')
        prob_h1, prob_h2, prob_h3 = params.get('dropout')
        
        # ENTRADA
        model_input = Input(shape=(ni,))
    
        # CAPAS COMUNES A TODAS LOS TARGETS
        x = Dense(n, activation=act)(model_input)
        x = Dropout(prob_h1)(x)
        x = Dense(n1, activation=act)(x)
        x = Dropout(prob_h2)(x)
        x = Dense(n2, activation=act)(x)
        x = Dropout(prob_h3)(x)
    
        # CAPAS ESPECIFICAS A CADA TARGET (1 a 10)
        y1 = Dense(units = 1, activation= 'sigmoid', name = 'output_1')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y2 = Dense(units = 1, activation= 'sigmoid', name = 'output_2')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y3 = Dense(units = 1, activation= 'sigmoid', name = 'output_3')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y4 = Dense(units = 1, activation= 'sigmoid', name = 'output_4')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y5 = Dense(units = 1, activation= 'sigmoid', name = 'output_5')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y6 = Dense(units = 1, activation= 'sigmoid', name = 'output_6')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y7 = Dense(units = 1, activation= 'sigmoid', name = 'output_7')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y8 = Dense(units = 1, activation= 'sigmoid', name = 'output_8')(Dropout(prob_h3)(Dense(n, activation=act)(x)))    
        y9 = Dense(units = 1, activation= 'sigmoid', name = 'output_9')(Dropout(prob_h3)(Dense(n, activation=act)(x)))
        y10 =Dense(units = 1, activation= 'sigmoid', name = 'output_10')(Dropout(prob_h3)(Dense(n, activation=act)(x)))

        # MODELO FINAL
        model = Model(inputs=model_input, outputs=[y1,y2,y3,y4,y5,y6,y7,y8,y9,y10])
        print(model.summary())
        return model
    
    def compilar(self, params):
        # set seed before compiling
        MTLDNN.set_seed(s=42)
        
        loss = params.get('loss')
        optimizer = Adam(params.get('l_rate'))
        metrics = params.get('metrics')
        self.model.compile(loss=loss, optimizer=optimizer, metrics = metrics)
    
    def entrenar(self, data, params):
        # data
        train, test = data
        x_train, y_train = train
        x_test, y_test = test
        
        #early stopping
        min_delta_val = params.get('min_delta')
        patience_val = params.get('patience')
        w = params.get('w')
        
        early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=min_delta_val,
                                   patience=patience_val,
                                   verbose=0,
                                   mode='min',
                                   restore_best_weights=True)
        # fit
        n_epochs = params.get('n_epochs')
        
        self.model.fit(x = x_train,
                  y = y_train,
                  epochs = n_epochs,
                  validation_data = (x_test,y_test),
                  callbacks = [early_stop],
                  class_weight = w,
                  verbose = 2)
        
    def predecir(self, X):
        
        # computar predicciones
        pred = self.model.predict(X)
            
        # obtener etiquetas predichas para cada output
        y0 = np.where(pred[0] > 0.5, 1,0)
        y1 = np.where(pred[1] > 0.5, 1,0)
        y2 = np.where(pred[2] > 0.5, 1,0)
        y3 = np.where(pred[3] > 0.5, 1,0)
        y4 = np.where(pred[4] > 0.5, 1,0)
        y5 = np.where(pred[5] > 0.5, 1,0)
        y6 = np.where(pred[6] > 0.5, 1,0)
        y7 = np.where(pred[7] > 0.5, 1,0)
        y8 = np.where(pred[8] > 0.5, 1,0)
        y9 = np.where(pred[9] > 0.5, 1,0)
        
        # obtener etiquetas para task_1
        y_pred = y0 | y1 | y2 | y3 | y4 | y5 | y6 | y7 | y8 | y9 
        
        return y_pred
    

In [30]:
params = {'dropout':[0.25,0.15,0.1],
          'act':'relu',
          'lb':0.005,
          'arq':(100,50,10,5),
          'w': None,
          'loss':BinaryCrossentropy(), 
          'l_rate':0.0001, 
          'metrics':[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.Accuracy()], 
          'min_delta':0.0001, 
          'patience':500,
          'n_epochs':1}

mi_modelo = MTLDNN(1281, params)
mi_modelo.compilar(params)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1281)]       0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          128200      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 100)          0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           5050        dropout[0][0]                    
______________________________________________________________________________________________

In [31]:
import data
datapath ="../../data/task_2.csv"
train_folds = data.load_data(datapath)


f0 = train_folds[0]
mi_modelo.entrenar(f0, params)

/home/vsabando/virtualenvs/tf_gpu/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Train on 3053 samples, validate on 764 samples
3053/3053 - 10s - loss: 6.6818 - output_1_loss: 0.6674 - output_2_loss: 0.6677 - output_3_loss: 0.6800 - output_4_loss: 0.6702 - output_5_loss: 0.6622 - output_6_loss: 0.6756 - output_7_loss: 0.6598 - output_8_loss: 0.6718 - output_9_loss: 0.6629 - output_10_loss: 0.6620 - output_1_precision: 0.0076 - output_1_recall: 0.0714 - output_1_accuracy: 0.0000e+00 - output_2_precision: 0.0286 - output_2_recall: 0.0377 - output_2_accuracy: 0.0000e+00 - output_3_precision: 0.0113 - output_3_recall: 0.1333 - output_3_accuracy: 0.0000e+00 - output_4_precision: 0.1724 - output_4_recall: 0.0202 - output_4_accuracy: 0.0000e+00 - output_5_precision: 0.0000e+00 - output_5_recall: 0.0000e+00 - output_5_accuracy: 0.0000e+00 - output_6_precision: 0.0694 - output_6_recall: 0.1156 - output_6_accuracy: 0.0000e+00 - output_7_precision: 0.0000e+00 - output_7_recall: 0.0000e+00 - output_7_accuracy: 0.0000e+00 - output_8_precision: 0.0463 - output_8_recall: 0.0474 -

In [32]:
train, test = f0
X_train, y_train = train
X_test, y_test = test

y_pred = mi_modelo.predecir(X_train)

In [1]:
y_test

NameError: name 'y_test' is not defined

In [35]:
y_pred.shape

(3053, 1)

In [43]:
importlib.reload(utils)

<module 'tensorflow_core.keras.utils' from '/home/vsabando/virtualenvs/tf_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/api/_v2/keras/utils/__init__.py'>